### Import Modules/Packages

In [17]:
import pandas as pd
import numpy as np

### Import Data (Train and Test)

In [18]:
df_house_train = pd.read_csv(
    r"train.csv",
    low_memory=False,
)

# drop columns with all NaN values
df_house_train.dropna(axis=1, how="all")


# limit to only residential properties
residential = ["FV", "RH", "RL", "RP", "RM"]
df_house_train = df_house_train[df_house_train["MSZoning"].isin(residential)]
print("Train Data Shape: ", df_house_train.shape)


Train Data Shape:  (1450, 81)


In [19]:
# Fill missing data with zero

def fill_missing_data(df: pd.DataFrame):
    for col_ in df.columns:
        if df[col_].dtype == "object":
            # fill 'NONE' for categorical features
            df[col_].fillna("NONE", inplace=True)
        else:
            # fill zero for numerical features
            df[col_].fillna(0, inplace=True)


fill_missing_data(df_house_train)


In [20]:
correlation = df_house_train.corr()
top_10_corr = correlation.nlargest(11, "SalePrice")["SalePrice"].index
cm = np.corrcoef(df_house_train[top_10_corr].values.T)


#### Building the Models

In [21]:
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler


In [22]:
# Define features to use in model 
Features = top_10_corr.copy().delete([0])


In [23]:
# Create copy of data set
train_df = df_house_train.copy()
# Filter to rows included in df_house_train set for train 
train_df = train_df[train_df.Id.isin(df_house_train.Id)]

# Set up Train and Validation Sets
    # test size = 20%
    # random state = 42

X_train, X_valid, y_train, y_valid = train_test_split(
    train_df[Features], train_df["SalePrice"], test_size=0.2, random_state=42
)
_, X_valid_with_id, _, _ = train_test_split(
    train_df.drop("SalePrice", axis=1),
    train_df["SalePrice"],
    test_size=0.2,
    random_state=42,
)
# Create copy of comb_df
test_df = df_house_train.copy()

X_test = test_df[Features]


#### XGBoost Model

In [24]:
from xgboost import XGBRegressor

In [25]:
XGmodel = XGBRegressor(n_estimators=3000, learning_rate=0.005)

In [26]:
XGmodel.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.005, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=3000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [27]:
print(f"Train Accuracy : {XGmodel.score(X_train,y_train)}")
print(f"Validation Accuracy : {XGmodel.score(X_valid,y_valid)}")

Train Accuracy : 0.9897838709073911
Validation Accuracy : 0.8419648622066889


In [28]:
XGdata = test_df
XGdata["SalePrice"] = XGmodel.predict(X_test)
XGdata = XGdata[["Id", "SalePrice"]]
XGdata


,Id,SalePrice
0,1,198742.765625
1,2,168695.468750
2,3,218233.843750
3,4,144132.546875
4,5,255934.203125
...,...,...
1455,1456,177297.015625
1456,1457,209995.906250
1457,1458,267713.125000
1458,1459,138333.468750


In [29]:
# Used for UI
import pickle
# Save the XGmodel to a file
with open('xgboost_model.pkl', 'wb') as file:
    pickle.dump(XGmodel, file)